<a href="https://colab.research.google.com/github/luciainnocenti/IncrementalLearning/blob/master/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import packages

In [0]:
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
from torch.utils.data import Subset, DataLoader

# Hyper-Parameters

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'
BATCH_SIZE = 256
NUM_WORKERS = 100
TASK_SIZE = 10
STEP_SIZE = 20       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down
LR = 1
NUM_EPOCHS = 30

# Define Network

In [0]:
resNet = models.resnet18(pretrained=False)
resNet = resNet.to(DEVICE)

# Define Data Loader

In [5]:
train_loader = torch.utils.data.DataLoader(
    Dataset(train=True),
    num_workers=NUM_WORKERS,
    batch_size=BATCH_SIZE
)

test_loader = torch.utils.data.DataLoader(
    Dataset(train=False),
    num_workers=NUM_WORKERS,
    batch_size=BATCH_SIZE
)

Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified


# Define training parameters

In [0]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(resNet.parameters(), lr=LR)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

# Train phase

In [7]:
test_indexes = []
for task in range(0, 100, TASK_SIZE):
  print("Task classes {} to {}".format(task, task + TASK_SIZE))

  train_indexes = train_loader.dataset.__getIndexesGroups__(task)
  test_indexes = test_indexes.append(test_loader.dataset.__getIndexesGroups__(task))

  train_dataset = Subset(Dataset(train=True), train_indexes)
  test_dataset = Subset(Dataset(train=False), test_indexes)
  resNet.fc = nn.Linear(512, task)

  for epoch in range(NUM_EPOCHS):
    for images, labels in train_loader:
    # Bring data over the device of choice
      images = images.to(DEVICE)
      labels = labels.to(DEVICE)
      print(labels)
      optimizer.zero_grad() # Zero-ing the gradients

      # Forward pass to the network
      outputs = resNet(images)

      loss = criterion(outputs, labels)

      loss.backward()  # backward pass: computes gradients
      optimizer.step() # update weights based on accumulated gradients
      print("At step ", str(task), " and at epoch = ", epoch, " the loss is = ", loss)
  scheduler.step() 

Task classes 0 to 10
Files already downloaded and verified
Files already downloaded and verified
tensor([19, 29,  0, 11,  1, 86, 90, 28, 23, 31, 39, 96, 82, 17, 71, 39,  8, 97,
        80, 71, 74, 59, 70, 87, 59, 84, 64, 52, 42, 64,  8, 17, 47, 65, 21, 22,
        81, 11, 24, 84, 78, 45, 49, 97, 56, 76, 11, 90, 89, 78, 73, 14, 87,  9,
        71,  6, 47, 20, 98, 47, 36, 55, 72, 43, 51, 35, 83, 33, 27, 53, 92, 50,
        15, 89, 36, 18, 89, 46, 33, 42, 39, 64, 75, 38, 23, 42, 66, 77, 49, 18,
        46, 15, 35, 69, 95, 83, 75, 99, 73, 93, 55, 39,  4, 97, 61, 93, 51, 69,
        56, 84, 59, 64, 94,  4, 11, 33, 68, 38, 20, 33, 34, 32, 46, 53, 88, 67,
        70, 64, 53, 64,  8, 96, 87, 30, 20, 30, 66, 19, 76, 87, 52, 62, 35, 63,
        40,  4, 99, 63, 74, 53, 26, 95, 48, 27, 33, 29, 39, 79, 32, 46, 64, 28,
        85, 32, 82, 78, 39, 54, 28, 66, 65, 72, 21, 64, 62, 72,  0, 44,  7, 12,
        19, 11, 31, 61, 79, 45, 81, 79, 98, 43, 46, 67, 80, 68, 74, 48, 81, 94,
        86, 69, 39, 73,

RuntimeError: ignored